### **Plant seedlings classification using CNN**

**Importing the required packages**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pickle
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import time
from sklearn.model_selection import KFold

**Mounting the google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR_TRAIN = "/content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/data/train"
DATA_DIR_TEST = "/content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/data/test"

In [ ]:
IMG_SIZE = 80

CATEGORIES = [
    "Black-grass",
    "Charlock",
    "Cleavers",
    "Common Chickweed",
    "Common wheat",
    "Fat Hen",
    "Loose Silky-bent",
    "Maize",
    "Scentless Mayweed",
    "Shepherds Purse",
    "Small-flowered Cranesbill",
    "Sugar beet"
]

**Create training data**

In [ ]:
training_data = []

def create_training_data():    
    for category in CATEGORIES:
        path = os.path.join(DATA_DIR_TRAIN, category)  # path to a specific category
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
            
create_training_data()

In [ ]:
print(len(training_data))

4790


In [ ]:
random.shuffle(training_data)

In [ ]:
X = []
Y = []

for features, label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = np.array(Y)

In [ ]:

X = X/255

In [ ]:
pickle_out = open(f"/content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/X_{IMG_SIZE}.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open(f"/content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/Y_{IMG_SIZE}.pickle", "wb")
pickle.dump(Y, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open(f"/content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/X_{IMG_SIZE}.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open(f"/content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/Y_{IMG_SIZE}.pickle", "rb")
Y = pickle.load(pickle_in)
pickle_in.close()

**Specifing run configurations and regularization constants. These are used to tweak the model.**


In [ ]:
# RUN CONFIGURATIONS
TEST = 21
CNN_ACT_F = "relu"
DNS_ACT_F = "tanh"
ARCH = f"CNN(64,128,256)_{CNN_ACT_F}_DENSE(128,64,32,12)_{DNS_ACT_F}"
IMG_SIZE = 80
N_SPLIT = 5
EPOCHS = 30
BATCH_SIZE = 50
VAL_SPLIT = 0.1

LR = 0.0095
MM = 0.9


# REGULARIZATIONS
DROPOUT_RATE = 0.5
LAMBDA_1 = 1e-5
LAMBDA_2 = 1e-6

In [ ]:
X = pickle.load(open(f"/content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/X_{IMG_SIZE}.pickle", "rb"))
Y = pickle.load(open(f"/content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/Y_{IMG_SIZE}.pickle", "rb"))

In [ ]:
SET_1 = f"{TEST} - {ARCH} - {IMG_SIZE}px_{EPOCHS}e_{BATCH_SIZE}bs_{int(VAL_SPLIT*100)}vs_REG_{int(DROPOUT_RATE*100)}do_{int(LAMBDA_1*1000000)}L1_{int(LAMBDA_2*1000000)}L2"
SET_2 = f"{TEST} - {IMG_SIZE}px_{EPOCHS}e_{BATCH_SIZE}bs_{int(VAL_SPLIT*100)}vs_REG_{int(DROPOUT_RATE*100)}do_{int(LAMBDA_1*1000000)}L1_{int(LAMBDA_2*1000000)}L2_{int(LR*10000)}LR_{int(MM*100)}M"
REPORT_NAME = SET_2

In [ ]:
    

MODEL_NAME = f"NN_GA_CNN_{int(time.time())}"

tensorboard = TensorBoard(log_dir=f"logs/{MODEL_NAME}")

def create_cnn_model():

  model = Sequential()
  model.add(Conv2D(64, (3,3), input_shape=(IMG_SIZE,IMG_SIZE,1)))  # input shape is IMG_SIZExIMG_SIZEx1
#     model.add(BatchNormalization(axis=3))
  model.add(Activation(CNN_ACT_F))
  model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
  model.add(Conv2D(128, (3,3)))
  model.add(Activation(CNN_ACT_F))
  model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
  model.add(Conv2D(256, (3,3)))
  model.add(Activation(CNN_ACT_F))
  model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(128, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Dropout(rate=DROPOUT_RATE))
    
  model.add(Flatten())
    
#     model.add(Dense(128))
  model.add(Dense(128, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
  model.add(Activation(DNS_ACT_F))
    
  model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
  model.add(Activation(DNS_ACT_F))
    
  model.add(Dense(32, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
  model.add(Activation(DNS_ACT_F))
    
  model.add(Dense(12))
  model.add(Activation("softmax"))
    
  optimizer = tf.keras.optimizers.SGD(lr=LR, momentum=MM, nesterov=False)
  model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
  return model

In [ ]:
report_dir = "/content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/reports"
if not os.path.exists(report_dir):
    os.makedirs(report_dir)
file = open(f"{report_dir}/{REPORT_NAME}.txt", "w")

In [ ]:
# initializing variables for kFold run and average accuracy
current_fold = 0
sum_acc = 0
avg_acc = 0

start = time.process_time()

for train_index, test_index in KFold(N_SPLIT).split(X):
    current_fold += 1
    
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    model = create_cnn_model()
    
#     earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VAL_SPLIT, callbacks=[tensorboard])
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(f"\nFOLD-{current_fold}: Loss={val_loss} , Accuracy={val_acc}\n")
    
    sum_acc += val_acc 
    file.write(f"{current_fold}-FOLD | Loss={round(val_loss,4)},\tAccuracy={round(val_acc,4)},\tAverage_Accuracy={round(sum_acc/current_fold,4)}\n")
    
    if(current_fold == N_SPLIT):
        avg_acc = round(sum_acc/current_fold,4)
    

avg_acc_line = f"\nAverage Accuracy : {round(avg_acc,4)}"

end = time.process_time()
time_taken = f"\nExecution Time\t : {round(end-start,4)}s"

print(avg_acc_line)
print(time_taken)

file.write(avg_acc_line)
file.write(time_taken)
    
file.close()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/30
69/69 [==============================] - 97s 1s/step - loss: 2.6199 - accuracy: 0.1343 - val_loss: 2.5815 - val_accuracy: 0.1302
Epoch 2/30
69/69 [==============================] - 95s 1s/step - loss: 2.5765 - accuracy: 0.1514 - val_loss: 2.5206 - val_accuracy: 0.1302
Epoch 3/30
69/69 [==============================] - 98s 1s/step - loss: 2.5330 - accuracy: 0.1560 - val_loss: 2.4927 - val_accuracy: 0.2083
Epoch 4/30
69/69 [==============================] - 95s 1s/step - loss: 2.5028 - accuracy: 0.1795 - val_loss: 2.4719 - val_accuracy: 0.2161
Epoch 5/30
69/69 [==============================] - 95s 1s/step - loss: 2.4834 - accuracy: 0.1969 - val_loss: 2.5014 - val_accuracy: 0.1771
Epoch 6/30
69/69 [==============================] - 98s 1s/step - loss: 2.4361 - accuracy: 0.2027 - val_loss: 2.3622 - val_accuracy: 0.2474
Epoch 7/30
69/69 [==============================] - 96s 1s/step - loss: 2.3626 - accuracy: 0.2497 - val_loss: 2.3216 - val_accuracy: 0.2604
Epoch 8/30
69/69 [==

In [ ]:
model.save(f"{report_dir}/{REPORT_NAME}.model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/MSC/NeuralNetwork/CNNAssignment/reports/21 - 80px_30e_50bs_10vs_REG_50do_10L1_1L2_95LR_90M.model/assets


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 78, 78, 64)        640       
_________________________________________________________________
activation_28 (Activation)   (None, 78, 78, 64)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 36, 36, 128)       73856     
_________________________________________________________________
activation_29 (Activation)   (None, 36, 36, 128)       0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 15, 15, 256)      